In [2]:
import torch
from torch.nn import functional as F
import ollama

In [3]:
model="deepseek-r1:7b"

In [26]:
import math
import re
from typing import Dict, List, Tuple

def calculate_path_distance(coords: Dict[str, List[float]], path: List[int]) -> float:
    """
    Calculate the total distance of a path through the given coordinates.
    
    Args:
        coords: Dictionary mapping node IDs to coordinate pairs.
        path: List of node IDs representing the order of visitation.
        
    Returns:
        Total Euclidean distance of the path.
    """
    total_distance = 0
    for i in range(len(path)):
        current_node = path[i]
        next_node = path[0] if i == len(path) - 1 else path[i + 1]
        
        # Fix the coordinate lookup to use the actual node IDs from the path
        x1, y1 = coords[str(current_node+1)]
        x2, y2 = coords[str(next_node+1)]
        
        # Calculate Euclidean distance
        distance = math.sqrt((x2 - x1)**2 + (y2 - y1)**2)
        total_distance += distance
    
    return total_distance

import ollama

def generate_solutions(prompt_dict, model="llama3", max_new_tokens=300, temperature=0.5, n_samples=20):
    """
    Generates responses using the Ollama API with a given model.
    """
    # Convert structured messages to a properly formatted chat prompt
    # formatted_prompt = "".join([msg["content"] for msg in prompt_dict])
    
    solutions = []
    for _ in range(n_samples):
        response = ollama.chat(
            model=model,
            messages=prompt_dict,
            options={"temperature": temperature, "num_predict": max_new_tokens}
        )
        solutions.append(response["message"]["content"])
    
    return solutions


       
def extract_trace(solution_text: str) -> List[int]:
    """
    Extract the first valid trace from the solution text.
    
    Args:
        solution_text: The generated solution text.
        
    Returns:
        List of integers representing the trace.
    """
    # Split the text into lines for line-by-line processing
    lines = solution_text.strip().split('\n')
    
    for line in lines:
        # Skip empty lines
        if not line.strip():
            continue
        
        # Look for patterns like "0,1,2,3,4" or "Trace 0,4,1,2,3"
        trace_patterns = [
            r'(\d+(?:,\s*\d+)+)',  # Basic comma-separated numbers
            r'[Tt]race\s*(\d+(?:,\s*\d+)+)',  # Prefixed with "Trace"
            r'trace\s*is\s*:\s*(\d+(?:,\s*\d+)+)',  # "trace is: 0,1,2,3"
        ]
        
        for pattern in trace_patterns:
            match = re.search(pattern, line)
            if match:
                trace_str = match.group(1)
                # Clean up the trace string and convert to integers
                try:
                    trace = [int(num.strip()) for num in trace_str.split(',')]
                    # Ensure we have a valid trace (at least 2 nodes)
                    if len(trace) >= 2:
                        return trace
                except ValueError:
                    # If conversion fails, continue to the next pattern
                    continue
    
    # If no valid trace is found, return an empty list
    return []


def test_model(conversation, model, coords,solution, max_iterations=20, n_samples=5):
    """
    Iteratively tests the model by generating multiple traces and evaluating their distances.
    
    Args:
        conversation: List of message dictionaries with role and content.
        tokenizer: The tokenizer for the model.
        model: The language model.
        coords: Dictionary of node coordinates.
        max_iterations: Maximum number of iterations before stopping.
    
    Returns:
        Updated conversation history with appended interactions.
    """
    tested_traces = set()
    best_distance = float("inf")
    best_trace = []
    iteration = 0
    no_improvement_counter = 0

    while iteration < max_iterations:
        total_distance = 0
        iteration += 1
        print(f"\nIteration {iteration}")
        
        # Generate 5 different solutions
        solutions = generate_solutions(
            conversation, 
            model, 
            temperature=0.5, 
            n_samples=n_samples, 
            max_new_tokens=1000
        )
        
        # Track the best solution from this batch
        batch_best_distance = float("inf")
        batch_best_trace = []
        
        # Evaluate each solution
        for i, solution_text in enumerate(solutions):
            trace = extract_answer(solution_text)
            print(trace)
            
            if not trace:
                print(f"  Solution {i+1}: No valid trace extracted")
                # print(solution_text)
                continue
                
            if len(trace) != len(set(trace)):
                print(f"  Solution {i+1}: Invalid trace (duplicate nodes)")
                continue

            if set(trace) != set(range(len(coords))):
                print(f"Wrong numbers in trace")
                continue
                
            # if tuple(trace) in tested_traces:
            #     print(f"  Solution {i+1}: Duplicate trace {trace}")
            #     continue
            
            # Calculate the distance
            try:
                distance = calculate_path_distance(coords, trace)
                total_distance += distance
                print(f"  Solution {i+1}: Trace {trace} with distance {distance:.2f}")
                
                # Update batch best
                if distance < batch_best_distance:
                    batch_best_distance = distance
                    batch_best_trace = trace
                
                # Add to tested traces
                tested_traces.add(tuple(trace))
                
            except KeyError as e:
                print(f"  Solution {i+1}: Error calculating distance for trace {trace} - {e}")
        
        
        print(f"Average distance of {total_distance/len(solutions)}")
        
        # Check if we found any valid traces in this batch
        if not batch_best_trace:
            print("No valid traces found in this batch. Stopping.")
            break
        
        # Check if the best trace from this batch improves on the overall best
        if batch_best_distance >= best_distance:
            no_improvement_counter += 1
            
        else:
            no_improvement_counter = 0
            best_distance = batch_best_distance
            best_trace = batch_best_trace
            
        if no_improvement_counter >= 3:
            print(f"No improvement over best distance {best_distance:.2f} on last 3 iterations. Stopping.")
            break
        
        if best_distance-.1 <= solution['distance']:
            break
        
        # Append new conversation turns
        trace_str = ", ".join(map(str, best_trace))

        conversation[0]['content'] += f'\n\n <trace>{trace_str},0<trace> \n lengthL \n {batch_best_distance:.2f}'
        # conversation.append({"role": "assistant", "content": trace_str})
        # conversation.append({
        #     "role": "user", 
        #     "content": f"Distance: {batch_best_distance:.2f}\nPlease find a trace with a shorter distance:"
        # })
        
        # del conversation[1:3]

    print(f"\nFinal best trace: {best_trace}")
    print(f"Final best distance: {best_distance:.2f}")
    
    solved = 0
    if best_distance-.1 <= solution['distance']:
        solved = 1
        print("Optimal Solution Found")
    
    return conversation, solved

def extract_answer(text):
    """
    Extracts the last occurrence of a list inside <trace> brackets (both <trace>list<trace> and <trace>list</trace>)
    and removes trailing 0 if present.
    """
    matches = re.findall(r"<trace>([\d,]+)<\/trace>|<trace>([\d,]+)<trace>", text)
    if not matches:
        return None
    
    # Extract the last non-empty match
    last_match = [m for match in matches for m in match if m][-1]
    numbers = last_match.split(",")
    
    # Remove trailing '0' if present
    if numbers and numbers[-1] == "0":
        numbers.pop()
    
    return [int(num) for num in numbers]

In [5]:
import json
with open("tsp_dataset_100_problems.json", "r") as f:
    problems = f.read()
problems = json.loads(problems)
problems['size_5'][0]['coordinates']

{'1': [63, -72],
 '2': [-94, 89],
 '3': [-30, -38],
 '4': [-43, -65],
 '5': [88, -74]}

In [27]:
with open("tsp_llm_prompts_with_matrix_think.json", "r") as f:
    
    prompts = f.read()
    
prompts = json.loads(prompts)
prompt = prompts['size_10'][0]['prompt']
prompt

[{'role': 'system',
  'content': 'You are solving a Traveling Salesperson Problem (TSP). Your goal is to generate an optimized route with a lower total distance than any previously provided solution.\n\n\n                   ## Problem Setup:\n\n                   The problem consists of the following nodes with coordinates:\n(1): (67, 35), (2): (-98, 70), (3): (41, -24), (4): (69, -74), (5): (-66, -33), (6): (-71, -73), (7): (90, 41), (8): (-61, -31), (9): (-28, 54), (10): (-47, 83).\n\n### Distance Matrix:\n Distance Matrix:\n       0    1    2    3    4    5    6    7    8    9\n 0:  0.0 168.7 64.5 109.0 149.4 175.2 23.8 144.0 96.9 123.7 \n 1: 168.7  0.0 167.8 220.5 107.9 145.5 190.2 107.6 71.8 52.6 \n 2: 64.5 167.8  0.0 57.3 107.4 122.2 81.4 102.2 104.1 138.5 \n 3: 109.0 220.5 57.3  0.0 141.1 140.0 116.9 136.9 160.6 195.2 \n 4: 149.4 107.9 107.4 141.1  0.0 40.3 172.7  5.4 94.9 117.5 \n 5: 175.2 145.5 122.2 140.0 40.3  0.0 197.3 43.2 134.1 157.8 \n 6: 23.8 190.2 81.4 116.9 172.7 197.

In [91]:
test = generate_solutions(prompt, model='deepseek-r1:7b', n_samples=1, max_new_tokens=5000)

In [92]:
print(test[0])

<think>
Alright, I need to solve this TSP problem with 10 nodes and find an optimized route that has a total distance lower than the previous best of 1216. The given traces are three different routes, so my new one must be unique and better.

First, I'll list out all the nodes and their coordinates:

1: (67,35)
2: (-98,70)
3: (41,-24)
4: (69,-74)
5: (-66,-33)
6: (-71,-73)
7: (90,41)
8: (-61,-31)
9: (-28,54)
10: (-47,83)

The distance matrix is provided. I'll need to calculate the distances between consecutive nodes in any potential route and sum them up.

Looking at the previous best trace: 0-7-5-3-1-4-8-6-9-2 with a total of 1216. So, my new trace needs to have a lower distance.

I think starting from 0 is standard, so I'll start there too. To minimize the total distance, it's important to visit nodes that are closer together first. Maybe grouping some nodes in one area before moving on could help reduce overall distance.

Let me consider possible clusters:

- Nodes near positive x: 7

In [28]:
prompts_5 = prompts['size_5']
problems_5 = problems['size_5']

correct = 0


for prompt_dict, problem_dict in zip(prompts_5, problems_5):
    
    prompt = prompt_dict["prompt"]
    coords = problem_dict["coordinates"]
    solution = problem_dict["solution"]
    
    results = test_model(prompt, "llama3.2:latest", coords, solution, n_samples=3)
    
    correct += results[1]
    
    



Iteration 1
None
  Solution 1: No valid trace extracted
[0, 2, 4, 3, 1, 0, 5]
  Solution 2: Invalid trace (duplicate nodes)
None
  Solution 3: No valid trace extracted
Average distance of 0.0
No valid traces found in this batch. Stopping.

Final best trace: []
Final best distance: inf

Iteration 1
None
  Solution 1: No valid trace extracted
[0, 4, 5, 3, 2, 1]
Wrong numbers in trace
[0, 3, 4, 2, 1]
  Solution 3: Trace [0, 3, 4, 2, 1] with distance 691.75
Average distance of 230.58247068579848

Iteration 2
[0, 4, 2, 3, 1]
  Solution 1: Trace [0, 4, 2, 3, 1] with distance 817.03
None
  Solution 2: No valid trace extracted
None
  Solution 3: No valid trace extracted
Average distance of 272.3429685504158

Iteration 3
None
  Solution 1: No valid trace extracted
[0, 3, 4, 2, 1, 5]
Wrong numbers in trace
[0, 4, 2, 3, 1]
  Solution 3: Trace [0, 4, 2, 3, 1] with distance 817.03
Average distance of 272.3429685504158

Iteration 4
[0, 4, 2, 3, 5, 1]
Wrong numbers in trace
[0, 2, 3, 4, 1]
  Solutio

ValueError: invalid literal for int() with base 10: ''

In [29]:
print(correct)

3


In [32]:
prompts_10 = prompts['size_10']
problems_10 = problems['size_10']

correct = 0


for prompt_dict, problem_dict in zip(prompts_10, problems_10):
    
    prompt = prompt_dict["prompt"]
    coords = problem_dict["coordinates"]
    solution = problem_dict["solution"]
    
    results = test_model(prompt, "llama3.2:latest", coords, solution, n_samples=20)
    
    correct += results[1]
    
    



Iteration 1
None
  Solution 1: No valid trace extracted
[0, 6, 2, 5, 1, 8, 9, 7, 4, 3]
  Solution 2: Trace [0, 6, 2, 5, 1, 8, 9, 7, 4, 3] with distance 1092.98
None
  Solution 3: No valid trace extracted
[0, 9, 5, 2, 7, 4, 3, 6, 8, 1]
  Solution 4: Trace [0, 9, 5, 2, 7, 4, 3, 6, 8, 1] with distance 1080.98
[0, 5, 1, 4, 9, 3, 8, 6, 7, 2]
  Solution 5: Trace [0, 5, 1, 4, 9, 3, 8, 6, 7, 2] with distance 1113.43
None
  Solution 6: No valid trace extracted
[0, 7, 9, 4, 3, 5, 1, 6, 2]
Wrong numbers in trace
None
  Solution 8: No valid trace extracted
[0, 1, 9, 5, 3, 4, 2, 6, 8, 7]
  Solution 9: Trace [0, 1, 9, 5, 3, 4, 2, 6, 8, 7] with distance 1092.84
None
  Solution 10: No valid trace extracted
[0, 7, 9, 8, 5, 4, 1, 6, 3, 2]
  Solution 11: Trace [0, 7, 9, 8, 5, 4, 1, 6, 3, 2] with distance 1165.96
None
  Solution 12: No valid trace extracted
[0, 7, 9, 8, 3, 5, 6, 4, 2, 1]
  Solution 13: Trace [0, 7, 9, 8, 3, 5, 6, 4, 2, 1] with distance 1064.45
None
  Solution 14: No valid trace extracted

KeyboardInterrupt: 

In [144]:
print(correct)

0
